# Wielkoskalowe metody pomiarowe w biologii molekularnej
## Metody selekcji cech - 2. Selekcja genów lub białek o ekspresji różnicującej dwie grupy próbek przy użyciu metody Recursive Feature Elimination
### Bartosz Sowul, Katarzyna Szmulewicz

## test: instalacja potrzebnych modułów

In [2]:
%matplotlib inline
!pip install matplotlib
!pip install pandas
!pip install numpy
!pip install scikit_learn

## test: ładowanie potrzebnych modułów

In [3]:
import pandas as pd

## Analiza wstępna na przykładowym zbiorze danych

In [4]:
def load_data(dataset):
    df = pd.read_csv(dataset, sep='\t')
    df = df.transpose()
    return df

df = load_data('./data/Leukemia/Leukemia.txt')
df.head()

,0,1,2,3,4,5,6,7,8,9,...,7120,7121,7122,7123,7124,7125,7126,7127,7128,7129
#Leukemia,#ALL B-cell#ALL T-cell#AML,AFFX-BioB-5_at,AFFX-BioB-M_at,AFFX-BioB-3_at,AFFX-BioC-5_at,AFFX-BioC-3_at,AFFX-BioDn-5_at,AFFX-BioDn-3_at,AFFX-CreX-5_at,AFFX-CreX-3_at,...,U48730_at,U58516_at,U73738_at,X06956_at,X16699_at,X83863_at,Z17240_at,L49218_f_at,M71243_f_at,Z78285_f_at
ALLB 01,1,-214,-153,-58,88,-295,-558,199,-176,252,...,185,511,-125,389,-37,793,329,36,191,-37
ALLB 04,1,-135,-114,265,12,-419,-585,158,-253,49,...,240,835,218,174,-110,627,170,-50,126,-91
ALLB 05,1,-106,-125,-76,168,-230,-284,4,-122,70,...,156,649,57,504,-26,250,314,14,56,-25
ALLB 07,1,-72,-144,238,55,-399,-551,131,-179,126,...,30,819,-178,151,-18,1140,482,10,369,-42


In [5]:
df.iloc[1:].head()

,0,1,2,3,4,5,6,7,8,9,...,7120,7121,7122,7123,7124,7125,7126,7127,7128,7129
ALLB 01,1,-214,-153,-58,88,-295,-558,199,-176,252,...,185,511,-125,389,-37,793,329,36,191,-37
ALLB 04,1,-135,-114,265,12,-419,-585,158,-253,49,...,240,835,218,174,-110,627,170,-50,126,-91
ALLB 05,1,-106,-125,-76,168,-230,-284,4,-122,70,...,156,649,57,504,-26,250,314,14,56,-25
ALLB 07,1,-72,-144,238,55,-399,-551,131,-179,126,...,30,819,-178,151,-18,1140,482,10,369,-42
ALLB 08,1,-413,-260,7,-2,-541,-790,-275,-463,70,...,289,629,-86,302,23,1799,446,59,781,20


In [6]:
df.rename(columns={0: 'class'}, inplace=True)
df.iloc[1:].describe()

,class,1,2,3,4,5,6,7,8,9,...,7120,7121,7122,7123,7124,7125,7126,7127,7128,7129
count,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,...,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0
unique,3.0,66.0,64.0,65.0,68.0,69.0,65.0,70.0,67.0,59.0,...,61.0,71.0,64.0,67.0,62.0,72.0,64.0,53.0,67.0,56.0
top,1.0,-62.0,-114.0,-50.0,132.0,-191.0,-284.0,-31.0,-141.0,-6.0,...,103.0,690.0,57.0,201.0,-62.0,893.0,208.0,0.0,333.0,-41.0
freq,38.0,2.0,3.0,3.0,2.0,2.0,3.0,2.0,3.0,3.0,...,3.0,2.0,3.0,2.0,2.0,1.0,3.0,5.0,2.0,3.0


In [7]:
df.iloc[1:][df.columns.difference(['class'])].head()

,1,2,3,4,5,6,7,8,9,10,...,7120,7121,7122,7123,7124,7125,7126,7127,7128,7129
ALLB 01,-214,-153,-58,88,-295,-558,199,-176,252,206,...,185,511,-125,389,-37,793,329,36,191,-37
ALLB 04,-135,-114,265,12,-419,-585,158,-253,49,31,...,240,835,218,174,-110,627,170,-50,126,-91
ALLB 05,-106,-125,-76,168,-230,-284,4,-122,70,252,...,156,649,57,504,-26,250,314,14,56,-25
ALLB 07,-72,-144,238,55,-399,-551,131,-179,126,-20,...,30,819,-178,151,-18,1140,482,10,369,-42
ALLB 08,-413,-260,7,-2,-541,-790,-275,-463,70,-169,...,289,629,-86,302,23,1799,446,59,781,20


In [8]:
df1 = df.iloc[1:]
df1.head()

,class,1,2,3,4,5,6,7,8,9,...,7120,7121,7122,7123,7124,7125,7126,7127,7128,7129
ALLB 01,1,-214,-153,-58,88,-295,-558,199,-176,252,...,185,511,-125,389,-37,793,329,36,191,-37
ALLB 04,1,-135,-114,265,12,-419,-585,158,-253,49,...,240,835,218,174,-110,627,170,-50,126,-91
ALLB 05,1,-106,-125,-76,168,-230,-284,4,-122,70,...,156,649,57,504,-26,250,314,14,56,-25
ALLB 07,1,-72,-144,238,55,-399,-551,131,-179,126,...,30,819,-178,151,-18,1140,482,10,369,-42
ALLB 08,1,-413,-260,7,-2,-541,-790,-275,-463,70,...,289,629,-86,302,23,1799,446,59,781,20


In [9]:
df1['class'].describe()

count     72.0
unique     3.0
top        1.0
freq      38.0
Name: class, dtype: float64

In [10]:
counts = df.iloc[1:, 0].value_counts()
counts

1.0    38
3.0    25
2.0     9
Name: class, dtype: int64

In [11]:
counts.idxmin()

2.0

In [12]:
counts[counts.idxmin()]

9

In [13]:
counts[counts > min(counts)]

1.0    38
3.0    25
Name: class, dtype: int64

In [14]:
df1['class'][df1['class'] != counts.idxmin()].describe()

count     63.0
unique     2.0
top        1.0
freq      38.0
Name: class, dtype: float64

In [15]:
df1['class'][df1['class'] != counts.idxmin()].value_counts()

1.0    38
3.0    25
Name: class, dtype: int64

In [16]:
df1[df1['class'] != counts.idxmin()].describe()

,class,1,2,3,4,5,6,7,8,9,...,7120,7121,7122,7123,7124,7125,7126,7127,7128,7129
count,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,...,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0
unique,2.0,58.0,56.0,57.0,60.0,60.0,58.0,61.0,58.0,54.0,...,54.0,62.0,57.0,59.0,54.0,63.0,57.0,48.0,59.0,52.0
top,1.0,-62.0,-114.0,-50.0,132.0,-191.0,-284.0,-222.0,-141.0,-6.0,...,103.0,690.0,57.0,201.0,-62.0,893.0,342.0,0.0,74.0,-2.0
freq,38.0,2.0,3.0,3.0,2.0,2.0,3.0,2.0,3.0,3.0,...,3.0,2.0,3.0,2.0,2.0,1.0,2.0,5.0,2.0,2.0


In [17]:
df.isnull().values.any()

False

In [18]:
df2 = df1[df1['class'] != counts.idxmin()]
df2.describe()

,class,1,2,3,4,5,6,7,8,9,...,7120,7121,7122,7123,7124,7125,7126,7127,7128,7129
count,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,...,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0,63.0
unique,2.0,58.0,56.0,57.0,60.0,60.0,58.0,61.0,58.0,54.0,...,54.0,62.0,57.0,59.0,54.0,63.0,57.0,48.0,59.0,52.0
top,1.0,-62.0,-114.0,-50.0,132.0,-191.0,-284.0,-222.0,-141.0,-6.0,...,103.0,690.0,57.0,201.0,-62.0,893.0,342.0,0.0,74.0,-2.0
freq,38.0,2.0,3.0,3.0,2.0,2.0,3.0,2.0,3.0,3.0,...,3.0,2.0,3.0,2.0,2.0,1.0,2.0,5.0,2.0,2.0


In [19]:
df2.head()

,class,1,2,3,4,5,6,7,8,9,...,7120,7121,7122,7123,7124,7125,7126,7127,7128,7129
ALLB 01,1,-214,-153,-58,88,-295,-558,199,-176,252,...,185,511,-125,389,-37,793,329,36,191,-37
ALLB 04,1,-135,-114,265,12,-419,-585,158,-253,49,...,240,835,218,174,-110,627,170,-50,126,-91
ALLB 05,1,-106,-125,-76,168,-230,-284,4,-122,70,...,156,649,57,504,-26,250,314,14,56,-25
ALLB 07,1,-72,-144,238,55,-399,-551,131,-179,126,...,30,819,-178,151,-18,1140,482,10,369,-42
ALLB 08,1,-413,-260,7,-2,-541,-790,-275,-463,70,...,289,629,-86,302,23,1799,446,59,781,20


In [20]:
df2 = df2.apply(pd.to_numeric)
df2.dtypes

class    float64
1        float64
2        float64
3        float64
4        float64
5        float64
6        float64
7        float64
8        float64
9        float64
10       float64
11       float64
12       float64
13       float64
14       float64
15       float64
16       float64
17       float64
18       float64
19       float64
20       float64
21       float64
22       float64
23       float64
24       float64
25       float64
26       float64
27       float64
28       float64
29       float64
          ...   
7100     float64
7101     float64
7102     float64
7103     float64
7104     float64
7105     float64
7106     float64
7107     float64
7108     float64
7109     float64
7110     float64
7111     float64
7112     float64
7113     float64
7114     float64
7115     float64
7116     float64
7117     float64
7118     float64
7119     float64
7120     float64
7121     float64
7122     float64
7123     float64
7124     float64
7125     float64
7126     float64
7127     float

In [21]:
df2.head()

,class,1,2,3,4,5,6,7,8,9,...,7120,7121,7122,7123,7124,7125,7126,7127,7128,7129
ALLB 01,1.0,-214.0,-153.0,-58.0,88.0,-295.0,-558.0,199.0,-176.0,252.0,...,185.0,511.0,-125.0,389.0,-37.0,793.0,329.0,36.0,191.0,-37.0
ALLB 04,1.0,-135.0,-114.0,265.0,12.0,-419.0,-585.0,158.0,-253.0,49.0,...,240.0,835.0,218.0,174.0,-110.0,627.0,170.0,-50.0,126.0,-91.0
ALLB 05,1.0,-106.0,-125.0,-76.0,168.0,-230.0,-284.0,4.0,-122.0,70.0,...,156.0,649.0,57.0,504.0,-26.0,250.0,314.0,14.0,56.0,-25.0
ALLB 07,1.0,-72.0,-144.0,238.0,55.0,-399.0,-551.0,131.0,-179.0,126.0,...,30.0,819.0,-178.0,151.0,-18.0,1140.0,482.0,10.0,369.0,-42.0
ALLB 08,1.0,-413.0,-260.0,7.0,-2.0,-541.0,-790.0,-275.0,-463.0,70.0,...,289.0,629.0,-86.0,302.0,23.0,1799.0,446.0,59.0,781.0,20.0


In [31]:
df2[df2.columns.difference([0])].head()

,1,2,3,4,5,6,7,8,9,10,...,7121,7122,7123,7124,7125,7126,7127,7128,7129,class
ALLB 01,-214.0,-153.0,-58.0,88.0,-295.0,-558.0,199.0,-176.0,252.0,206.0,...,511.0,-125.0,389.0,-37.0,793.0,329.0,36.0,191.0,-37.0,1.0
ALLB 04,-135.0,-114.0,265.0,12.0,-419.0,-585.0,158.0,-253.0,49.0,31.0,...,835.0,218.0,174.0,-110.0,627.0,170.0,-50.0,126.0,-91.0,1.0
ALLB 05,-106.0,-125.0,-76.0,168.0,-230.0,-284.0,4.0,-122.0,70.0,252.0,...,649.0,57.0,504.0,-26.0,250.0,314.0,14.0,56.0,-25.0,1.0
ALLB 07,-72.0,-144.0,238.0,55.0,-399.0,-551.0,131.0,-179.0,126.0,-20.0,...,819.0,-178.0,151.0,-18.0,1140.0,482.0,10.0,369.0,-42.0,1.0
ALLB 08,-413.0,-260.0,7.0,-2.0,-541.0,-790.0,-275.0,-463.0,70.0,-169.0,...,629.0,-86.0,302.0,23.0,1799.0,446.0,59.0,781.0,20.0,1.0


In [33]:
df2.iloc[:, 0].head()

ALLB 01    1.0
ALLB 04    1.0
ALLB 05    1.0
ALLB 07    1.0
ALLB 08    1.0
Name: class, dtype: float64